In [10]:
import tensorflow as tf

# Tensorflow核心教程

##  计算图

您可能会认为TensorFlow Core程序由两个独立部分组成：  
- 构建计算图。
- 运行计算图。

甲计算图形是一系列排列成节点的图形TensorFlow操作。我们来构建一个简单的计算图。每个节点采用零个或多个张量作为输入，并产生张量作为输出。一种类型的节点是一个常数。像所有TensorFlow常数一样，它不需要任何输入，它输出一个内部存储的值。我们可以创建两个浮点式传感器node1 ，node2如下所示：

In [11]:
node1 = tf.constant(3.0,tf.float32)
node2 = tf.constant(4.0) # also tf.float32 implicitly
print(node1,node2)

Tensor("Const_2:0", shape=(), dtype=float32) Tensor("Const_3:0", shape=(), dtype=float32)


请注意，打印节点不会输出值3.0，4.0正如您所期望的那样。相反，它们是在评估时分别产生3.0和4.0的节点。要实际评估节点，我们必须在会话中运行计算图。会话封装了TensorFlow运行时的控制和状态。
下面的代码创建一个Session对象，然后调用其run方法运行足够的计算图来评价node1和node2。通过在会话中运行计算图如下：

In [12]:
sess = tf.Session()
print(sess.run([node1,node2]))

[3.0, 4.0]


我们可以通过将Tensor节点与操作相结合来构建更复杂的计算（操作也是节点）。例如，我们可以添加我们的两个常量节点并生成一个新的图，如下所示：

In [13]:
node3 = tf.add(node1, node2)
print("node3: ", node3)
print("sess.run(node3): ",sess.run(node3))

node3:  Tensor("Add_1:0", shape=(), dtype=float32)
sess.run(node3):  7.0


TensorFlow提供了一个名为TensorBoard的实用程序，可以显示计算图的图片。这是一个屏幕截图，显示TensorBoard如何可视化图形：  
<img src="https://raw.githubusercontent.com/mokundong/images/master/t_1.jpg" alt="GitHub" title="GitHub,Social Coding" width="150" height="150" />
就这样，这个图并不是特别有趣，因为它总是产生一个恒定的结果。可以将图形参数化为接受外部输入，称为占位符。一个占位符是一个承诺后提供一个值。

In [14]:
a = tf.placeholder(tf.float32)
b = tf.placeholder(tf.float32)
adder_node = a + b  # + provides a shortcut for tf.add(a, b)

前面的三行有点像一个函数或一个lambda，其中我们定义了两个输入参数（a和b），然后对它们进行一个操作。我们可以使用feed_dict参数来指定多个输入的图表来指定为这些占位符提供具体值的Tensors：

In [15]:
print(sess.run(adder_node, {a: 3, b:4.5}))
print(sess.run(adder_node, {a: [1,3], b: [2, 4]}))

7.5
[ 3.  7.]


在TensorBoard中，图形如下所示：  
<img src="https://raw.githubusercontent.com/mokundong/images/master/t_2.jpg" alt="GitHub" title="GitHub,Social Coding" width="150" height="150" /> 
我们可以通过添加另一个操作来使计算图更加复杂。例如:

In [16]:
add_and_triple = adder_node * 3.
print(sess.run(add_and_triple, {a: 3, b:4.5}))

22.5


前面的计算图在TensorBoard中将如下所示：  
<img src="https://raw.githubusercontent.com/mokundong/images/master/t_3.jpg" alt="GitHub" title="GitHub,Social Coding" width="150" height="150" />  
在机器学习中，我们通常会想要一个可以接受任意输入的模型，比如上面的一个。为了使模型可训练，我们需要能够修改图形以获得具有相同输入的新输出。 变量允许我们向图中添加可训练的参数。它们的构造类型和初始值：

In [17]:
W = tf.Variable([.3], tf.float32)
b = tf.Variable([-.3], tf.float32)
x = tf.placeholder(tf.float32)
linear_model = W * x + b

常数被调用时初始化tf.constant，其值永远不会改变。相比之下，调用时，变量不会被初始化tf.Variable。要初始化TensorFlow程序中的所有变量，必须显式调用特殊操作，如下所示：

In [18]:
init = tf.global_variables_initializer()
sess.run(init)

重要的是实现initTensorFlow子图的一个句柄，初始化所有的全局变量。在我们调用之前sess.run，变量未初始化。
既然x是占位符，我们可以同时评估linear_model几个值， x如下所示：

In [19]:
print(sess.run(linear_model, {x:[1,2,3,4]}))

[ 0.          0.30000001  0.60000002  0.90000004]


我们创建了一个模型，但是我们不知道它有多好。为了评估培训数据的模型，我们需要一个y占位符来提供所需的值，我们需要编写一个损失函数。
损失函数测量当前模型与提供的数据之间的距离。我们将使用线性回归的标准损失模型，其将当前模型和提供的数据之间的三角形的平方相加。linear_model - y创建一个向量，其中每个元素都是对应的示例的错误增量。我们打电话tf.square给这个错误。然后，我们求和所有平方误差，创建一个单一的标量，使用tf.reduce_sum以下方法抽象出所有示例的错误：

In [20]:
y = tf.placeholder(tf.float32)
squared_deltas = tf.square(linear_model - y)
loss = tf.reduce_sum(squared_deltas)
print(sess.run(loss,{x:[1,2,3,4],y:[0,-1,-2,-3]}))

23.66


我们可以手动重新分配的值提高这W和b为-1和1变量的值，完美初始化为提供的价值 tf.Variable，但可以使用操作等来改变tf.assign。例如， W=-1并且b=1是我们的模型的最佳参数。我们可以改变W，并 b因此：

In [21]:
fixW = tf.assign(W,[-1.])
fixb = tf.assign(b,[1.])
sess.run([fixW, fixb])
print(sess.run(loss, {x:[1,2,3,4], y:[0,-1,-2,-3]}))

0.0


### tf.train API

机器学习的完整讨论超出了本教程的范围。然而，TensorFlow提供了优化器，缓慢地更改每个变量，以便最大程度地减少损失函数。最简单的优化器是梯度下降。它根据相对于该变量的损失导数的大小修改每个变量。通常，手动计算符号导数是乏味且容易出错的。因此，TensorFlow可以使用该函数自动生成仅给出模型描述的导数tf.gradients。为了简单起见，优化器通常为您做这个。例如:

In [22]:
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

In [23]:
sess.run(init)# reset values to incorrect defaults.
for i in range(1000):
    sess.run(train,{x:[1,2,3,4],y:[0,-1,-2,-3]})
    
print(sess.run([W,b]))

[array([-0.9999969], dtype=float32), array([ 0.99999082], dtype=float32)]


现在我们已经完成了实际的机器学习！虽然这样做简单的线性回归并不需要太多的TensorFlow核心代码，但更复杂的模型和方法将数据输入到模型中需要更多的代码。因此，TensorFlow为常见的模式，结构和功能提供了更高级别的抽象。我们将在下一节中学习如何使用其中的一些抽象。

### 完成程序

In [32]:
import tensorflow as tf
import numpy as np

# Model parameters
W = tf.Variable([.3],tf.float32)
b = tf.Variable([-.3],tf.float32)
# Model input and output
x = tf.placeholder(tf.float32)
linear_model = W * x + b
y = tf.placeholder(tf.float32)
# loss
loss = tf.reduce_sum(tf.square(linear_model - y)) # sum of the squares
# optimizer
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)
# trainning data
x_train = [1,2,3,4]
y_train = [0,-1,-2,-3]
# trainning loop
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init) # reset values to wrong
for i in range(1000):
    sess.run(train,{x:x_train,y:y_train})
    if i % 100 == 0:
            print(i,sess.run([W, b, loss], {x:x_train, y:y_train}))
# evaluate training accuracy
curr_W, curr_b, curr_loss  = sess.run([W, b, loss], {x:x_train, y:y_train})
print("W: %s b: %s loss: %s"%(curr_W, curr_b, curr_loss))

0 [array([-0.21999997], dtype=float32), array([-0.456], dtype=float32), 4.0181446]
100 [array([-0.84270465], dtype=float32), array([ 0.53753263], dtype=float32), 0.14287975]
200 [array([-0.95284992], dtype=float32), array([ 0.86137295], dtype=float32), 0.012838207]
300 [array([-0.98586655], dtype=float32), array([ 0.95844597], dtype=float32), 0.0011535464]
400 [array([-0.99576342], dtype=float32), array([ 0.98754394], dtype=float32), 0.00010365112]
500 [array([-0.99873012], dtype=float32), array([ 0.99626648], dtype=float32), 9.3124017e-06]
600 [array([-0.99961936], dtype=float32), array([ 0.99888098], dtype=float32), 8.3645574e-07]
700 [array([-0.99988592], dtype=float32), array([ 0.9996646], dtype=float32), 7.514916e-08]
800 [array([-0.99996579], dtype=float32), array([ 0.99989945], dtype=float32), 6.7539112e-09]
900 [array([-0.99998969], dtype=float32), array([ 0.99996972], dtype=float32), 6.1273298e-10]
W: [-0.9999969] b: [ 0.99999082] loss: 5.69997e-11


这个更复杂的程序仍然可以在TensorBoard中可视化   
![GIthub](https://raw.githubusercontent.com/mokundong/images/master/t_4.jpg)

### tf.contrib.learn

**tf.contrib.learn** 是一个高级_TensorFlow_库，简化了机器学习的机制，其中包括：  
- 运行训练循环
- 运行评估循环
- 管理数据集
- 管理喂养 

tf.contrib.learn定义了许多常见的模型。

**基本用法**  
注意线性回归程序变得简单得多 tf.contrib.learn：

In [33]:
import tensorflow as tf
import numpy as np
 
# Declare list of features. We only have one real-valued feature. There are many
# other types of columns that are more complicated and useful.
features = [tf.contrib.layers.real_valued_column("x", dimension=1)]
 
# An estimator is the front end to invoke training (fitting) and evaluation
# (inference). There are many predefined types like linear regression,
# logistic regression, linear classification, logistic classification, and
# many neural network classifiers and regressors. The following code
# provides an estimator that does linear regression.
estimator = tf.contrib.learn.LinearRegressor(feature_columns=features)
 
# TensorFlow provides many helper methods to read and set up data sets.
# Here we use `numpy_input_fn`. We have to tell the function how many batches
# of data (num_epochs) we want and how big each batch should be.
x = np.array([1., 2., 3., 4.])
y = np.array([0., -1., -2., -3.])
input_fn = tf.contrib.learn.io.numpy_input_fn({"x":x}, y, batch_size=4,
                                              num_epochs=1000)
 
# We can invoke 1000 training steps by invoking the `fit` method and passing the
# training data set.
estimator.fit(input_fn=input_fn, steps=1000)
 
# Here we evaluate how well our model did. In a real example, we would want
# to use a separate validation and testing data set to avoid overfitting.
print(estimator.evaluate(input_fn=input_fn))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fa1c60439b0>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/tmpyoly21c7'}
Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer sup

### 自定义模型

tf.contrib.learn不会锁定您的预定义模型。假设我们想创建一个没有内置到TensorFlow中的自定义模型。我们仍然可以保留数据集，饲养，培训等的高层抽象 tf.contrib.learn。为了说明，我们将展示如何实现我们自己的等效模型，以LinearRegressor使用我们对较低级别TensorFlow API的了解。  

要定义一个适合的自定义模型tf.contrib.learn，我们需要使用 tf.contrib.learn.Estimator。tf.contrib.learn.LinearRegressor实际上是一个子类tf.contrib.learn.Estimator。Estimator我们只是提供Estimator一个功能model_fn来告诉 tf.contrib.learn它如何评估预测，训练步骤和损失，而不是分类 。代码如下：

In [34]:
import numpy as np
import tensorflow as tf
# Declare list of features, we only have one real-valued feature
def model(features, labels, mode):
  # Build a linear model and predict values
  W = tf.get_variable("W", [1], dtype=tf.float64)
  b = tf.get_variable("b", [1], dtype=tf.float64)
  y = W*features['x'] + b
  # Loss sub-graph
  loss = tf.reduce_sum(tf.square(y - labels))
  # Training sub-graph
  global_step = tf.train.get_global_step()
  optimizer = tf.train.GradientDescentOptimizer(0.01)
  train = tf.group(optimizer.minimize(loss),
                   tf.assign_add(global_step, 1))
  # ModelFnOps connects subgraphs we built to the
  # appropriate functionality.
  return tf.contrib.learn.ModelFnOps(
      mode=mode, predictions=y,
      loss=loss,
      train_op=train)
 
estimator = tf.contrib.learn.Estimator(model_fn=model)
# define our data set
x = np.array([1., 2., 3., 4.])
y = np.array([0., -1., -2., -3.])
input_fn = tf.contrib.learn.io.numpy_input_fn({"x": x}, y, 4, num_epochs=1000)
 
# train
estimator.fit(input_fn=input_fn, steps=1000)
# evaluate our model
print(estimator.evaluate(input_fn=input_fn, steps=10)) 

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fa1c5d37828>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/tmpm18i08nd'}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tmpm18i08nd/model.ckpt.
INFO:tensorflow:loss = 0.872889114814, step = 1
INFO:tensorflow:global_step/sec: 1084.29
INFO:tensorflow:loss = 0.021092450254, step = 101 (0.096 sec)
INFO:tensorflow:global_step/sec: 1280.25
INFO:tenso